In [392]:
# START
# Generate the initial population
# Compute fitness
# REPEAT
#     Selection
#     Crossover
#     Mutation
#     Compute fitness
# UNTIL population has converged
# STOP

In [586]:
import numpy as np
rng = np.random.default_rng()

In [587]:
# helper functions to generate strategies
def generate_random_strat():
    allocation = np.zeros(10)
    for i in range(10):
        allocation[i] = rng.random()
    return normalise_strat(allocation)
    
def normalise_strat(allocation):
    allocation = np.floor(100 / allocation.sum() * allocation)
    # make weights sum to 100
    remainder = 100 - int(allocation.sum())
    for i in range(remainder):
        j = rng.integers(low=0, high=9, size=1)
        allocation[j] += 1
    return allocation

def custom_strat(snippet, start):
    strat = np.zeros(10)
    strat[start:start+len(snippet)] = snippet
    return strat

In [589]:
class strategy:
    def __init__(self):
        self.allocation = None
        self.score = 0
    
    def random_allocation(self):
        self.allocation = generate_random_strat()

    def compete(strat_A, strat_B):
        count = 0
        last_winner = None
        winner = -1
        for i in range(10):
            # compare armies
            if strat_A.allocation[i] > strat_B.allocation[i]:
                strat_A.score += i + 1
                winner = 0
            elif strat_A.allocation[i] < strat_B.allocation[i]:
                strat_B.score += i + 1
                winner = 1
            
            # 3 strikes rule
            if winner != last_winner:
                count = 1
                last_winner = winner
            else:
                count += 1
                if count == 3:
                    reward = 0
                    for j in range(i+1, 10):
                        reward += j + 1
                    if winner == 0:
                        strat_A.score += reward
                    else:
                        strat_B.score += reward
                    break
    
    def confirm_strat(self):
        assert len(self.allocation) == 10
        assert sum(self.allocation) == 100
    
    def print_info(self):
        print("Strategy: ", self.allocation, " Score: ", self.score)

def mutate(strat):
        child = strategy()
        tmp = generate_random_strat()
        child.allocation = normalise_strat(strat.allocation * (70 / 100) + tmp * (30 / 100))
        return child

def crossover(strat_A, strat_B):
        child_A = strategy()
        child_A.allocation = normalise_strat(np.concatenate((strat_A.allocation[0:3], strat_B.allocation[3:])))
        child_B = strategy()
        child_B.allocation = normalise_strat(np.concatenate((strat_B.allocation[0:3], strat_A.allocation[3:])))
        return child_A, child_B

# def crossover(strat_A, strat_B):
#         child_A = strategy()
#         child_A.allocation = normalise_strat(np.concatenate((strat_A.allocation[0:3], strat_B.allocation[3:])))
#         child_B = strategy()
#         child_B.allocation = normalise_strat(np.concatenate((strat_B.allocation[0:3], strat_A.allocation[3:])))
#         return child_A, child_B

def update_fitness(population):
    for i in range(len(population)):
        population[i].score = 0
    for i in range(len(population)):
        for j in range(i + 1, len(population)):
            population[i].compete(population[j])

def initalise_pop(size):
    population = []
    for i in range(size):
        strat = strategy()
        strat.random_allocation()
        population.append(strat)
    return population


In [590]:
population = initalise_pop(2000)

# special solutions

#uniform allocation
uniform_strat = strategy()
uniform_strat.allocation = np.ones(10) * 10
population.append(uniform_strat)

# head heavy allocation
strat_1 = strategy()
strat_1.allocation = np.zeros(10)
strat_1.allocation[0:3] = np.array([33, 33, 34])
population.append(strat_1)

# proportional allocation
strat_2 = strategy()
prop = np.array([i for i in range(1,11)])
strat_2.allocation = normalise_strat(prop)
population.append(strat_2)

# more balanced strategy
strat_3 = strategy()
strat_3.allocation = custom_strat(np.array([35, 30, 29, 0, 2, 0, 2, 2]), 2)
population.append(strat_3)

# print a small range of strategies
for i in range(len(population) - 10, len(population)):
    print(population[i].allocation, sum(population[i].allocation))

[18. 13.  4.  5.  8. 11. 14.  5.  7. 15.] 100.0
[ 3. 11.  4. 17.  5. 15. 18.  9.  5. 13.] 100.0
[12. 13.  3.  2. 14.  5.  9. 18.  9. 15.] 100.0
[11.  4.  3. 15. 11. 12.  8.  8. 14. 14.] 100.0
[15.  0. 10.  9.  3.  2. 16. 15. 14. 16.] 100.0
[16.  7.  7.  5. 13.  3. 13. 11. 13. 12.] 100.0
[10. 10. 10. 10. 10. 10. 10. 10. 10. 10.] 100.0
[33. 33. 34.  0.  0.  0.  0.  0.  0.  0.] 100.0
[ 1.  5.  5.  7. 10. 12. 12. 14. 16. 18.] 100.0
[ 0.  0. 35. 30. 29.  0.  2.  0.  2.  2.] 100.0


In [591]:
def run_generation(population):
    # make strategies compete
    update_fitness(population)
    population.sort(key=lambda x: x.score, reverse=True)

    # print top 25 strategies
    for i in range(10):
        print("Strategy: ", population[i].allocation, " Score: ", population[i].score)

    # print bottom 5 strategies
    for i in range(len(population) - 5, len(population)):
        print("Strategy: ", population[i].allocation, " Score: ", population[i].score)
    
    # selection
    selected = population[0:len(population)//10]
    print("Selected: ", len(selected))

    # mutation
    mutations = []
    length = len(selected)
    for i in range(length):
        mutations.append(mutate(population[i]))
    print("Mutations: ", len(mutations))

    # crossover
    children = []
    length = len(selected)
    for i in range(30):
        for j in range(i + 1, 30):
            child_A, child_B = crossover(population[i], population[j])
            children.append(child_A)
            children.append(child_B)
    print("Children: ", len(children))

    print("New population: ", len(population))
    return selected + mutations + children 

In [ ]:
winners = []

In [731]:
# population.append(strat_1)
# population.append(strat_3)
# strat_4 = strategy()
# strat_4.allocation = np.array([1, 15, 51, 19, 12, 0, 2, 0, 0, 0])
# population.append(strat_4)
# strat_5 = strategy()
# strat_5.allocation = np.array([0, 0, 33, 27, 27, 0, 1, 4, 8, 0])
# population.append(strat_5)
# strat_7 = strategy()
# strat_7.allocation = np.array([2, 2, 2, 31, 31, 32, 0, 0, 0, 0])
# population.append(strat_7)
population = run_generation(population)
winners.append(population[0])

# [ 6. 11. 37. 14. 22.  7.  3.  0.  0.  0.]
# [ 2. 14. 43. 19. 20.  2.  0.  0.  0.  0.]

Strategy:  [11. 26. 25. 25.  1.  3.  5.  2.  2.  0.]  Score:  44780
Strategy:  [10. 12. 40. 25.  2.  3.  4.  2.  2.  0.]  Score:  44359
Strategy:  [11. 25. 24. 24.  1.  4.  4.  4.  3.  0.]  Score:  44092
Strategy:  [10. 12. 39. 24.  2.  3.  4.  3.  3.  0.]  Score:  43820
Strategy:  [11. 25. 24. 24.  3.  3.  3.  4.  3.  0.]  Score:  43535
Strategy:  [11. 25. 24. 26.  2.  2.  4.  3.  3.  0.]  Score:  43467
Strategy:  [10. 12. 39. 24.  1.  4.  4.  4.  2.  0.]  Score:  43263
Strategy:  [10. 12. 40. 24.  1.  3.  4.  4.  2.  0.]  Score:  43068
Strategy:  [11. 12. 35. 21. 12.  6.  3.  0.  0.  0.]  Score:  42957
Strategy:  [11. 13. 38. 25.  2.  2.  4.  2.  3.  0.]  Score:  42947
Strategy:  [13. 15. 16. 14. 11. 13.  6.  3.  6.  3.]  Score:  8830
Strategy:  [17. 18. 19. 17.  9.  9.  1.  2.  8.  0.]  Score:  7871
Strategy:  [19. 19. 22. 16.  1.  4.  8.  4.  4.  3.]  Score:  7168
Strategy:  [16. 10. 21. 16. 11. 12.  5.  3.  6.  0.]  Score:  6166
Strategy:  [11. 27. 22. 15.  1.  3.  8.  5.  3.  5.]

In [732]:
for i in winners:
    i.print_info()

Strategy:  [11. 34. 47.  1.  1.  2.  0.  2.  1.  1.]  Score:  18739
Strategy:  [ 1 15 51 19 12  0  2  0  0  0]  Score:  37682
Strategy:  [ 1. 18. 38. 17. 18.  3.  3.  2.  0.  0.]  Score:  26580
Strategy:  [ 4. 14. 41. 24. 13.  2.  1.  1.  0.  0.]  Score:  16785
Strategy:  [ 1. 16. 50. 19. 12.  0.  2.  0.  0.  0.]  Score:  38468
Strategy:  [ 0. 18. 38. 22. 21.  1.  0.  0.  0.  0.]  Score:  18009
Strategy:  [ 4. 15. 39. 14. 19.  5.  3.  1.  0.  0.]  Score:  36261
Strategy:  [ 0. 19. 40. 15. 18.  5.  3.  0.  0.  0.]  Score:  22253
Strategy:  [ 6. 12. 35. 17. 20.  7.  3.  0.  0.  0.]  Score:  15006
Strategy:  [ 0. 19. 42. 17. 18.  3.  1.  0.  0.  0.]  Score:  22244
Strategy:  [ 0. 17. 41. 18. 20.  3.  1.  0.  0.  0.]  Score:  38441
Strategy:  [ 2. 13. 42. 19. 20.  4.  0.  0.  0.  0.]  Score:  33776
Strategy:  [ 2. 14. 43. 19. 20.  2.  0.  0.  0.  0.]  Score:  19163
Strategy:  [ 0. 17. 43. 20. 19.  1.  0.  0.  0.  0.]  Score:  39320
Strategy:  [ 4. 17. 34. 21. 21.  3.  0.  0.  0.  0.]  Scor

In [733]:
population = run_generation(winners)

Strategy:  [33. 33. 34.  0.  0.  0.  0.  0.  0.  0.]  Score:  4381
Strategy:  [33. 33. 34.  0.  0.  0.  0.  0.  0.  0.]  Score:  4381
Strategy:  [22. 33. 42.  2.  1.  0.  0.  0.  0.  0.]  Score:  3898
Strategy:  [22. 34. 43.  0.  0.  0.  0.  1.  0.  0.]  Score:  3843
Strategy:  [19. 30. 46.  3.  1.  1.  0.  0.  0.  0.]  Score:  3809
Strategy:  [22. 33. 42.  1.  2.  0.  0.  0.  0.  0.]  Score:  3767
Strategy:  [14. 28. 43. 10.  2.  2.  1.  0.  0.  0.]  Score:  3728
Strategy:  [14. 28. 48.  4.  2.  3.  0.  1.  0.  0.]  Score:  3672
Strategy:  [13. 27. 27. 16.  8.  3.  6.  0.  0.  0.]  Score:  3563
Strategy:  [13. 27. 27. 16.  8.  3.  6.  0.  0.  0.]  Score:  3563
Strategy:  [ 9. 25. 30.  9.  6.  8.  5.  2.  1.  5.]  Score:  1236
Strategy:  [11. 18. 18. 11. 16.  9.  5.  7.  5.  0.]  Score:  1203
Strategy:  [15. 21. 22. 11.  7.  8.  8.  4.  4.  0.]  Score:  1093
Strategy:  [11. 18. 18.  8. 16.  8.  6.  8.  3.  4.]  Score:  1039
Strategy:  [26. 16. 19.  9.  7.  7.  4.  5.  1.  6.]  Score:  

In [581]:
winners = []
for i in range(100):
    population.append(strat_1)
    population.append(strat_3)
    strat_4 = strategy()
    strat_4.allocation = np.array([1, 15, 51, 19, 12, 0, 2, 0, 0, 0])
    population.append(strat_4)
    strat_5 = strategy()
    strat_5.allocation = np.array([0, 0, 33, 27, 27, 0, 1, 4, 8, 0])
    population.append(strat_5)
    strat_6 = strategy()
    strat_6.allocation = np.array([1, 15, 51, 19, 12, 0, 2, 0, 0, 0])
    population.append(strat_6)
    strat_7 = strategy()
    strat_7.allocation = np.array([0, 0, 0, 33, 33, 34, 0, 0, 0, 0])
    population.append(strat_7)
    population = run_generation(population)
    
    if i % 10 == 0:
        winners.append(population[0])
        
print(winners)

Strategy:  [11. 34. 47.  1.  1.  2.  0.  2.  1.  1.]  Score:  53887
Strategy:  [ 7. 29. 57.  2.  2.  1.  1.  0.  1.  0.]  Score:  52704
Strategy:  [ 8. 28. 58.  1.  2.  1.  2.  0.  0.  0.]  Score:  52116
Strategy:  [ 8. 29. 56.  1.  1.  1.  1.  1.  1.  1.]  Score:  51617
Strategy:  [10. 26. 58.  2.  2.  0.  0.  1.  1.  0.]  Score:  49449
Strategy:  [ 7. 32. 53.  2.  2.  0.  0.  3.  0.  1.]  Score:  49296
Strategy:  [ 8. 28. 59.  2.  1.  0.  0.  1.  1.  0.]  Score:  47999
Strategy:  [ 9. 26. 59.  1.  1.  0.  1.  2.  1.  0.]  Score:  47988
Strategy:  [ 7. 36. 51.  2.  1.  0.  1.  0.  1.  1.]  Score:  47972
Strategy:  [ 9. 30. 54.  1.  0.  1.  3.  2.  0.  0.]  Score:  47335
Strategy:  [ 1.  8. 38. 14. 17. 14.  4.  4.  0.  0.]  Score:  3633
Strategy:  [ 2.  7. 35. 14. 17. 12.  8.  4.  0.  1.]  Score:  2224
Strategy:  [ 0.  0. 35. 30. 29.  0.  2.  0.  2.  2.]  Score:  514
Strategy:  [ 0  0 33 27 27  0  1  4  8  0]  Score:  254
Strategy:  [ 0  0  0 33 33 34  0  0  0  0]  Score:  147
Selected

KeyboardInterrupt: 

In [584]:
for i in winners:
    i.print_info()

Strategy:  [11. 34. 47.  1.  1.  2.  0.  2.  1.  1.]  Score:  18739
Strategy:  [ 1 15 51 19 12  0  2  0  0  0]  Score:  37682
Strategy:  [ 1. 18. 38. 17. 18.  3.  3.  2.  0.  0.]  Score:  26580
Strategy:  [ 4. 14. 41. 24. 13.  2.  1.  1.  0.  0.]  Score:  16785
Strategy:  [ 1. 16. 50. 19. 12.  0.  2.  0.  0.  0.]  Score:  38468
Strategy:  [ 0. 18. 38. 22. 21.  1.  0.  0.  0.  0.]  Score:  18009


In [433]:
print(len(population))

1004


In [403]:
population = initalise_pop(1000000)

# special solutions

#uniform allocation
uniform_strat = strategy()
uniform_strat.allocation = np.ones(10) * 10
population.append(uniform_strat)

# head heavy allocation
strat_1 = strategy()
strat_1.allocation = np.zeros(10)
strat_1.allocation[0:3] = np.array([33, 33, 34])
population.append(strat_1)

# proportional allocation
strat_2 = strategy()
prop = np.array([i for i in range(1,11)])
strat_2.allocation = normalise_strat(prop)
population.append(strat_2)

# more balanced strategy
strat_3 = strategy()
strat_3.allocation = custom_strat(np.array([35, 30, 29, 0, 2, 0, 2, 2]), 2)
population.append(strat_3)

# # print a small range of strategies
# for i in range(len(population) - 10, len(population)):
#     print(population[i].allocation, sum(population[i].allocation))

# make strategies compete
update_fitness(population)
population.sort(key=lambda x: x.score, reverse=True)

# print top 25 strategies
for i in range(25):
    print("Strategy: ", population[i].allocation, " Score: ", population[i].score)

KeyboardInterrupt: 

In [404]:
# print top 25 strategies
population.sort(key=lambda x: x.score, reverse=True)

for i in range(25):
    print("Strategy: ", population[i].allocation, " Score: ", population[i].score)

Strategy:  [ 7. 17. 22. 12. 12. 15.  2.  0. 11.  2.]  Score:  44280990
Strategy:  [ 6. 19. 19. 14.  2.  1. 15. 19.  1.  4.]  Score:  43226460
Strategy:  [ 5.  4. 18. 17. 15.  0. 13. 10. 14.  4.]  Score:  42188202
Strategy:  [13. 16. 14. 13. 11.  5.  3.  7.  5. 13.]  Score:  39916366
Strategy:  [ 8. 10. 16. 10. 17.  8. 11. 17.  0.  3.]  Score:  38556359
Strategy:  [13.  9. 12. 13. 15. 14.  9.  8.  7.  0.]  Score:  38317976
Strategy:  [ 8. 12. 20. 11. 14.  2.  2.  3.  6. 22.]  Score:  36955865
Strategy:  [10.  1. 17. 15. 14.  1.  7. 15. 13.  7.]  Score:  35526515
Strategy:  [ 3. 18. 16.  7. 13.  7. 16.  8.  5.  7.]  Score:  34899044
Strategy:  [12. 15. 19.  6.  4.  0.  4. 17. 15.  8.]  Score:  34235473
Strategy:  [12. 16. 13.  2. 17.  8. 10.  4.  4. 14.]  Score:  33944838
Strategy:  [15. 15. 13.  6.  4.  6. 16. 12.  4.  9.]  Score:  33443817
Strategy:  [10. 28. 13. 10.  3.  0. 13.  4.  7. 12.]  Score:  33049142
Strategy:  [10. 13. 13.  7. 12.  7. 14. 11. 12.  1.]  Score:  32894051
Strate

In [399]:
# debugging

# strat_3.compete(strat_1)
# original = strat_3

# print(strat_3.score, strat_1.score)
# strat_3.print_info()
# strat_1.print_info()

# strat_3.mutate()
# strat_3.print_info()
# original.print_info()

# child_A, child_B = crossover(strat_1, strat_3)
# child_A.print_info()
# child_B.print_info()

In [440]:
def add_one(x):
    return x + x

In [441]:
x = [1, 2]

In [444]:
x = add_one(x)
print(x)

[1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2]
